In [11]:
import numpy as np

def print_info(a):
    """ Print the content of an array, and its metadata. """
    
    txt = f"""
dtype\t{a.dtype}
ndim\t{a.ndim}
shape\t{a.shape}
strides\t{a.strides}
    """

    print(a)
    print(txt)

# NumPy views and copies

- Operations that only require changing the metadata always do so, and return a **view**
- Operations that cannot be executed by changing the metadata create a new memory block, and return a **copy**

In [12]:
x = np.arange(12).reshape(3, 4).copy()
print_info(x)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

dtype	int64
ndim	2
shape	(3, 4)
strides	(32, 8)
    


# Views

Operations that only require changing the metadata always do so, and return a **view**

In [13]:
# slice
y = x[0::2, 1::2]
print_info(y)

[[ 1  3]
 [ 9 11]]

dtype	int64
ndim	2
shape	(2, 2)
strides	(64, 16)
    


A view shares the same memory block as the original array. 

In [14]:
z = x.reshape(1, 12)
print_info(z)

[[ 0  1  2  3  4  5  6  7  8  9 10 11]]

dtype	int64
ndim	2
shape	(1, 12)
strides	(96, 8)
    


CAREFUL: Modifying the view **changes the original array** and all other views of that array as well!

##### in place operations

In [15]:
y += 100
print_info(y)

[[101 103]
 [109 111]]

dtype	int64
ndim	2
shape	(2, 2)
strides	(64, 16)
    


In [16]:
print_info(x)
print_info(z)

[[  0 101   2 103]
 [  4   5   6   7]
 [  8 109  10 111]]

dtype	int64
ndim	2
shape	(3, 4)
strides	(32, 8)
    
[[  0 101   2 103   4   5   6   7   8 109  10 111]]

dtype	int64
ndim	2
shape	(1, 12)
strides	(96, 8)
    


Functions that take an array as an input should **avoid modifying it in place!***

Always make a copy or be super extra clear in the docstring.

In [18]:
def robust_log(x, cte=1e-10):
    """ Compute the log of the elements of an array.
    
    Values that are equal to 0.0 in `x` are substituted with a tiny constant `cte`
    to avoid a divide-by-zero warning, and `-inf` values in the output arrays.
    """
    x[x == 0] = cte
    return np.log(x)


In [28]:
a = np.array([[0.3, 0.01], [0, 1]])
a

array([[0.3 , 0.01],
       [0.  , 1.  ]])

In [29]:
np.log(a)

/tmp/ipykernel_33111/176755284.py:1: RuntimeWarning: divide by zero encountered in log
  np.log(a)


array([[-1.2039728 , -4.60517019],
       [       -inf,  0.        ]])

In [27]:
robust_log(a)

array([[ -1.2039728 ,  -4.60517019],
       [-23.02585093,   0.        ]])

In [23]:
a = np.array([[0.3, 0.01], [0, 1]])
b = a[1, :]
b

array([0., 1.])

In [24]:
robust_log(b)

array([-23.02585093,   0.        ])

In [30]:
b

array([1.e-10, 1.e+00])

In [25]:
a

array([[3.e-01, 1.e-02],
       [1.e-10, 1.e+00]])

Better make a copy!

In [ ]:
def robust_log(x, cte=1e-10):
    """ Compute the log of the elements of an array.
    
    Values that are equal to 0.0 in `x` are substituted with a tiny constant `cte`
    to avoid a divide-by-zero warning, and `-inf` values in the output arrays.
    """
    x = x.copy()
    x[x == 0] = cte
    return np.log(x)

In [ ]:
a = np.array([[0.3, 0.01], [0, 1]])
b = a[1, :]

#robust_sqrt(a)

In [ ]:
a  # what is the output?   
# b

# Copies

- Operations that cannot be executed by changing the metadata create a new memory block, and return a **copy**

- How to find out view or copy?

Choosing row, columns, or individual elements of an array by giving explicitly their indices (a.k.a "fancy indexing") it's an operation that in general cannot be executed by changing the metadata alone.

Therefore, **fancy indexing always returns a copy**.

In [ ]:
x = np.arange(12).reshape(3, 4).copy()
print_info(x)

In [ ]:
#print(x)
z = x[[0, 0, 2], [1, 0, 3]]
# Can you guess what's z equal to?

print_info(z)

In [ ]:
z += 1000
print_info(z)

# the original array is unchanged => not a view!
print_info(x)

**Views** are created, when you use other strides to read your data. Slicing and regular indexing allows that, as you know how many byte steps you need to take to get the data.

**Fancy indexing** does not allow that, because the data you are asking **cannot** be obtained by just changing the strides. Thus, numpy needs to create a **copy** of it in memory.